# Cumulative distribution functions

In [ ]:
import sys
sys.path.append('lib')

In [ ]:
from typing import List, Tuple
from collections import Counter
import bisect

In [ ]:
import numpy as np
import pandas as pd

import nsfg

In [ ]:
import seaborn as sns
from IPython.core.pylabtools import figsize
sns.set_theme()
figsize(10, 6)

## The limits of PMFs

In [ ]:
live = nsfg.read_live_fem_preg()

In [ ]:
live.birthcat.value_counts().sort_index()

Remove the na values from `totalwgt_lb`

In [ ]:
live[live.totalwgt_lb.notna()].birthcat.value_counts().sort_index()

In [ ]:
# lets remove them
live.dropna(subset=('totalwgt_lb',), inplace=True)

In [ ]:
live.birthcat.value_counts().sort_index()

And compute the distribution of birth weight for first babies and others.

In [ ]:
p = sns.histplot(
    data=live,
    x='totalwgt_lb',
    binwidth=0.2,
    stat='probability',
    hue='birthcat',
    multiple='dodge'
)
p.set(
    xlabel = 'Weight (lbs)',
    title = 'Distribution of birth weights for first born babies and others'
);
p.get_legend().set_title('Birth category');

We can plot the PMFs on the same scale, but it is hard to see if there is a difference.

This problems can be mitigated by binning the data; that is, dividing the range of values into non-overlapping intervals and counting the number of values in each bin. Binning can be useful, but it is tricky to get the size of the bins right. If they are big enough to smooth out noise, they might also smooth out useful information.

## Percentiles

`percentile_rank` computes the fraction of `scores` less than or equal to `your_score`.

In [ ]:
def percentile_rank(scores: List[int], your_score: int) -> int:
    '''
    Returns the percentage of values less than or equal to you score
    '''
    count = 0
    for score in scores:
        if score <= your_score:
            count += 1
    percentile_rank = 100 * count / len(scores)
    return percentile_rank

If this is the list of scores.

In [ ]:
t = [55, 66, 77, 88, 99]

And you got the 88, your percentile rank is 80.

In [ ]:
percentile_rank(t, 88)

`percentile` takes a percentile rank and computes the corresponding percentile. 

In [ ]:
def percentile(scores, rank):
    scores.sort()
    for score in scores:
        if percentile_rank(scores, score) >= rank:
            return score

The median is the 50th percentile, which is 77.

In [ ]:
percentile(t, 50)

Here's a more efficient way to compute percentiles.

In [ ]:
def percentile(scores, rank):
    scores.sort()
    index = rank * (len(scores)-1) // 100
    return scores[index]

Let's hope we get the same answer.

In [ ]:
percentile(t, 50)

However, `percentile` should not be sorting the scores - its inefficient and qualifies as a side effect. Instead it should be up to the client to keep maintain its scores in sorted order

In [ ]:
def percentile(scores, rank):
    index = rank * (len(scores)-1) // 100
    return scores[index]

In [ ]:
t.sort()

In [ ]:
percentile(t, 50)

The Cumulative Distribution Function (CDF) is almost the same as `percentile_rank`.  The only difference is that the result is 0-1 instead of 0-100.

In [ ]:
def eval_cdf(sample: List[int], x: int) -> float:
    count = 0.0
    for value in sample:
        if value <= x:
            count += 1

    prob = count / len(sample)
    return prob

In this list

In [ ]:
t = [1, 2, 2, 3, 5]

We can evaluate the CDF for various values:

In [ ]:
values = range(0, 6)
for value in values:
    print(f'cdf({value}) = {eval_cdf(t, value)}')

Here's an example using real data, the distribution of pregnancy length for live births.

In [ ]:
p = sns.ecdfplot(
    data=live,
    x='totalwgt_lb'
)
p.set(
    xlabel = 'Weight (in lbs)'
);

In [ ]:
p = sns.ecdfplot(
    data=live,
    x='totalwgt_lb',
    hue='birthcat'
)
p.set(
    xlabel = 'Birth weight (in lbs)'
);
p.get_legend().set_title('Birth category');

## Implementation

Any empirical CDF implementation has to be able to answer two queries:

- `percentile`: given a value x, what percentage of distribution is at or below x
- `quantile`: given a percentage p, return a value x such that p % of the distribution is at or below x

Lets use are simple array of numbers to illustrate this

In [ ]:
t

The key to implementating this is to have the values in sorted order and the probabilities are the cumulative values of the normalized frequencies.

We will use the aggregation functionality of the `Counter` class to compute our values and frequencies

In [ ]:
counter = Counter(t)
counter

return as a list of tuples sorted by value (not frequency)

In [ ]:
sorted(counter.items())

split into separate lists of values and frequencies

In [ ]:
values, freqs = zip(*sorted(Counter(t).items()))
print(f'Values: {values}, Frequencies: {freqs}')

Take the cumulative sum of frequencies

In [ ]:
probs = np.cumsum(freqs, dtype=np.float64)
print(probs)

And normalize them so they are expressed as a proportion of the largest value

In [ ]:
probs /= probs[-1]

In [ ]:
list(zip(values, np.round(probs, 2)))

So you can see that 80% (or 4/5) of `t` is less than or equal to 3, and conversely 3 is the value that is equal to or larger than 80% of `t`

Now because are values are in sorted order we can use bisection to find the position of any given value and use that index to find the corresponding probablity

In [ ]:
# bisect returns the position after its index value, so subtract 1
for x in values:
    idx = bisect.bisect(values, x)
    print(f'{x} -> Pos: {idx}, Prob: {probs[idx - 1]}')

If the value doesn't exist `bisect` returns the index of the value immediately before it

In [ ]:
for x in (4, 6):
    idx = bisect.bisect(values, x)
    print(f'{x} -> Pos: {idx}, Prob: {probs[idx - 1]}')

We use `bisect_left` for mapping probabilities back to values, which returns the index of where to

In [ ]:
bisect.bisect_left(probs, 0.1)

In [ ]:
for p in np.linspace(0.1, 1, 10):
    idx = bisect.bisect_left(probs, p)
    print(f'{p:0.2f} -> Pos: {idx}, Val: {values[idx]}')

To vectorize these operations we can use `np.searchsorted`

In [ ]:
indices = np.searchsorted(values, [1, 2, 3], side='right')
probs[indices]

We can now create our initial implementation

In [ ]:
type(probs)

In [ ]:
class Cdf:
    
    @classmethod
    def from_hist(cls, values, freqs):
        # convert frequencies to probabilities
        probs = np.cumsum(freqs, dtype=np.float64)
        # and normalize
        probs /= probs[-1]
        # call constructor
        return cls(np.asarray(values), probs)
    
    @classmethod
    def from_seq(cls, x):
        # x may contain duplicates
        values, freqs = zip(*sorted(Counter(x).items()))
        return cls.from_hist(values, freqs)
    
    def __init__(self, xs: np.ndarray, ps: np.ndarray):
        # values
        self.xs = xs
        # cumulative probabilities
        self.ps = ps
        
    
    def prob(self, x) -> float:
        """
        Returns CDF(x), the probability that corresponds to value x.

        Args:
            x: number

        Returns:
            float probability
        """
        if x < self.xs[0]:
            return 0
        # find x in our values
        index = bisect.bisect(self.xs, x)
        # return the corresponding probability
        return self.ps[index-1]
    
    def probs(self, xs: np.array) -> np.array:
        """
        Gets probabilities for a sequence of values.

        xs: any sequence that can be converted to NumPy array

        returns: NumPy array of cumulative probabilities
        """
        index = np.searchsorted(self.xs, xs, side='right')
        ps = self.ps[index-1]
        # anything less than our smallest value is zero
        ps[xs < self.xs[0]] = 0
        return ps
    
    def value(self, p: np.float64):
        """
        Returns InverseCDF(p), the value that corresponds to probability p.

        Args:
            p: number in the range [0, 1]

        Returns:
            number value
        """
        if p < 0 or p > 1:
            raise ValueError('Probability p must be in range [0, 1]')

        index = bisect.bisect_left(self.ps, p)
        return self.xs[index]
    
    def values(self, ps) -> np.array:
        """
        Returns InverseCDF(p), the value that corresponds to probability p.

        Args:
            ps: NumPy array of numbers in the range [0, 1]

        Returns:
            NumPy array of values
        """

        ps = np.asarray(ps)
        
        if np.any(ps < 0) or np.any(ps > 1):
            raise ValueError('Probability p must be in range [0, 1]')

        indices = np.searchsorted(self.ps, ps, side='left')
        return self.xs[indices]
    
    def percentile(self, p: np.int64) -> np.number:
        """
        Returns the value that corresponds to percentile p (between 0 and 100).

        Args:
            p: number in the range [0, 100]

        Returns:
            number value
        """
        return self.value(p / 100)
    
    def rank(self, x) -> np.number:
        """
        Returns the percentile rank of the value x.

        x: potential value in the CDF

        returns: percentile rank in the range 0 to 100
        """
        return self.prob(x) * 100
    
    def sample(self, n: int):
        """
        Returns a list of n values chosen at random from the cdf.
        
        n: int length of the sample
        returns: NumPy array
        """
        
        # n values between 0 and 1
        ps = np.random.random(n)
        # get the values they correspond to
        return self.values(ps)
    
    def mean(self) -> float:
        """
        Computes the mean of a CDF.
        
        sum(x * diff(p))

        Returns:
            float mean
        """
        old_p = 0
        total = 0
        for x, new_p in zip(self.xs, self.ps):
            # compute dp
            p = new_p - old_p
            total += p * x
            old_p = new_p
        return total
    
    def ci(self, percentage=90) -> Tuple[float, float]:
        """Computes the central credible interval.

        If percentage=90, computes the 90% CI.

        Args:
            percentage: float between 0 and 100

        Returns:
            sequence of two floats, low and high
        """
        prob = (1 - percentage / 100) / 2
        return self.value(prob), self.value(1 - prob)
    
    def items(self):
        """
        Returns a sorted sequence of (value, probability) pairs.
        """
        a = self.ps
        # shift probabilities one place to the right
        # e.g [0, 0.1, 0.2, 0.3] becomes [0.3, 0, 0.1, 0.2]
        b = np.roll(a, 1)
        b[0] = 0
        return zip(self.xs, a-b)
    
    @property
    def series(self) -> pd.Series:
        '''
        Return our value and cumulative probabilities as a pandas series
        '''
        return pd.Series(data=self.ps, index=self.xs)
    
    def copy(self):
        return self.__class__(self.xs.copy(), self.ps.copy())
    
    def compliment(self):
        '''
        Returns 1-CDF
        '''
        return self.__class__(self.xs.copy(), 1-self.ps)
    
    def as_dataframe(self) -> pd.DataFrame:
        return pd.DataFrame(dict(
            x=self.xs,
            p=self.ps
        ))
    
    def __len__(self):
        return len(self.xs)

In [ ]:
cdf = Cdf.from_seq(live.prglngth)

`Cdf` provides `Prob`, which evaluates the CDF; that is, it computes the fraction of values less than or equal to the given value.  For example, 94% of pregnancy lengths are less than or equal to 41.

In [ ]:
live.prglngth.mean()

`Value` evaluates the inverse CDF; given a fraction, it computes the corresponding value.  For example, the median is the value that corresponds to 0.5.

In [ ]:
cdf.value(0.5)

In general, CDFs are a good way to visualize distributions.  They are not as noisy as PMFs, and if you plot several CDFs on the same axes, any differences between them are apparent.

In [ ]:
p = sns.lineplot(
    x=cdf.xs,
    y=cdf.ps,
    drawstyle='steps-post'
);
p.set(
    xlabel = 'Pregancy length (weeks)'
);

Lets look at the birth weights

In [ ]:
cdf = Cdf.from_seq(live.totalwgt_lb)

Again, the median is the 50th percentile.

In [ ]:
cdf.percentile(50)

The interquartile range is the interval from the 25th to 75th percentile.

In [ ]:
(cdf.percentile(25), cdf.percentile(75))

We can use the CDF to look up the percentile rank of a particular value.  For example, my daughter was 8.2 pounds at birth, which is near the 77th percentile.

In [ ]:
cdf.rank(8.2)

If we draw a random sample from the observed weights and map each weigh to its percentile rank.

In [ ]:
sample = np.random.choice(live.totalwgt_lb, 100, replace=True)
ranks = [cdf.rank(x) for x in sample]

The resulting list of ranks should be approximately uniform from 0-1.

In [ ]:
rank_cdf = Cdf.from_seq(ranks)

In [ ]:
p = sns.lineplot(
    x = rank_cdf.xs,
    y = rank_cdf.ps,
    drawstyle='steps-post'
)
p.set(
    xlabel = 'Percentile rank',
    ylabel = 'CDF'
);

That observation is the basis of `Cdf.sample`, which generates a random sample from a Cdf.

In [ ]:
resample = Cdf.from_seq(cdf.sample(1000))

In [ ]:
resample.as_dataframe()

Compare this with the original

In [ ]:
weights = np.linspace(0, 16, len(cdf))
df = pd.DataFrame({
    'weight': weights,
    'source': cdf.probs(weights),
    'sampled': resample.probs(weights)
})

In [ ]:
df

In [ ]:
df_long = df.melt(
    id_vars = 'weight',
    value_vars = ['source', 'sampled'],
    value_name = 'probability',
    var_name = 'cdf'
)
df_long

In [ ]:
p = sns.lineplot(
    data=df_long,
    x='weight',
    y='probability',
    hue='cdf'
)
p.set(
    xlabel = 'Pregancy length (in weeks)',
    ylabel = 'CDF'
);

This confirms that the random sample has the same distribution as the original data.

## Complimentary CDF

## Exercises

**Exercise:** How much did you weigh at birth? Using the NSFG data (all live births), compute the distribution of birth weights and use it to find your percentile rank. If you were a first baby, find your percentile rank in the distribution for first babies. Otherwise use the distribution for others. If you are in the 90th percentile or higher, call your mother back and apologize.

In [ ]:
# Solution goes here
first_cdf = Cdf.from_seq(live.totalwgt_lb[live.birthcat == 'firsts'])

In [ ]:
first_cdf.rank(9)

In [ ]:
other_cdf = Cdf.from_seq(live.totalwgt_lb[live.birthcat == 'others'])

In [ ]:
other_cdf.rank(9)

**Exercise:** The numbers generated by `numpy.random.random` are supposed to be uniform between 0 and 1; that is, every value in the range should have the same probability.

Generate 1000 numbers from `numpy.random.random` and plot their PMF.  What goes wrong?

Now plot the CDF. Is the distribution uniform?

In [ ]:
# Solution goes here
t = np.random.random(1000)

In [ ]:
# the pmf would be a mess
p = sns.histplot(
    t,
    binwidth=0.01
);

In [ ]:
# Solution goes here
cdf = Cdf.from_seq(t)

In [ ]:
p = sns.lineplot(
    x=cdf.xs,
    y=cdf.ps,
    drawstyle='steps-post'
)
p.set(
    xlabel = 'U(x)',
    ylabel = 'cdf(x)'
);